In [1]:
!pip install snowflake[ml]==0.8.0

     |████████████████████████████████| 450kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 34.6MB/s eta 0:00:01
     |████████████████████████████████| 122kB 62.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 60.7MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 46.6MB/s eta 0:00:01
     |████████████████████████████████| 409kB 54.5MB/s eta 0:00:01
     |████████████████████████████████| 41.0MB 53.5MB/s eta 0:00:01
     |████████████████████████████████| 133kB 49.2MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 52.1MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 46.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 9.4MB/s  eta 0:00:01
     |████████████████████████████████| 18.3MB 21.8MB/s eta 0:00:01
     |████████████████████████████████| 174kB 56.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 14.3MB/s eta 0:00:01
     |████████████████████████████████| 747kB 53.6MB/s eta 0:0

     |████████████████████████████████| 2.1MB 44.7MB/s eta 0:00:01
     |████████████████████████████████| 450kB 49.0MB/s eta 0:00:01
     |████████████████████████████████| 512kB 47.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 10.7MB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 40.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 12.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 11.8MB/s eta 0:00:01
     |████████████████████████████████| 174kB 56.5MB/s eta 0:00:01
     |████████████████████████████████| 112kB 39.3MB/s eta 0:00:01
     |████████████████████████████████| 143kB 58.6MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 27.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 12.0MB/s eta 0:00:01
     |████████████████████████████████| 307kB 55.5MB/s eta 0:00:01
     |████████████████████████████████| 122kB 57.5MB/s eta 0:00:01
     |████████████████████████████████| 307kB 59.7MB/s eta 0:00:01

ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 4.19.0 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.6.0 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.7.3 which is incompatible.
ERROR: jsonschema-path 0.3.2 has requirement referencing<0.32.0,>=0.28.0, but you'll have referencing 0.33.0 which is incompatible.
ERROR: snowflake-connector-python 3.11.0 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.2.2 which is incompatible.
ERROR: snowflake-snowpark-python 1.18.0 has requirement cloudpickle!=2.1.0,!=2.2.0,<=2.2.1,>=1.6.0; python_version < "3.11", but you'll have cloudpickle 3.0.0 which is incompatible.
ERROR: botocore 1.34.131 has requirement urllib3<1.27,>=1.25.4; python_version < "3.10", but you'll have urllib3 2.2.2 which is incompatible.
You s

In [1]:
from snowflake.snowpark import Session
from snowflake.ml.registry import Registry as model_registry
import time

In [2]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWINSIGHT"

In [3]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=REGISTRY_DATABASE_NAME,
                         schema=REGISTRY_SCHEMA_NAME, 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [44]:
session = get_session()
# session.close()
# session

In [67]:
def fetch_models():
    session = get_session()
    registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )
    df = registry.show_models()
    return df

def fetch_model(model_name):
    session = get_session()
    registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )
    df = registry.get_model(model_name)
    return df

def fetch_model_query(model_name):
    session = get_session()
#     registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )
#     df = registry.get_model(model_name)
    df = session.sql("SHOW MODELS LIKE 'SAMPLE_MODEL_7' IN SCHEMA MODEL_REGISTRY_SNOWSIGHT.SNOWINSIGHT")

    return df

In [63]:
start_time = time.time()
df =fetch_models()
end_time = time.time()
print("time taken =", end_time - start_time)
df

time taken = 0.5465891361236572


,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-06-04 03:21:29.166000-07:00,SAMPLE_MODEL,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN1,"[""RUN1""]"
1,2024-06-04 04:47:08-07:00,SAMPLE_MODEL_10,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN4,"[""RUN4""]"
2,2024-06-04 03:26:19.863000-07:00,SAMPLE_MODEL_6,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN1,"[""RUN1"",""RUN2""]"
3,2024-06-04 03:45:33.469000-07:00,SAMPLE_MODEL_7,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN3,"[""RUN3""]"
4,2024-06-04 03:47:48.600000-07:00,SAMPLE_MODEL_8,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN3,"[""RUN3"",""RUN4""]"
5,2024-06-04 03:51:57.010000-07:00,SAMPLE_MODEL_9,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN4,"[""RUN4""]"
6,2024-06-05 05:50:06.224000-07:00,SAMPLE_MODEL_TEST_121,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN1,"[""RUN1""]"
7,2024-05-30 03:54:17.224000-07:00,SAMPLE_SKLEARN_MODEL,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
8,2024-05-31 02:58:30.562000-07:00,SAMPLE_SKLEARN_MODEL_2,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
9,2024-05-31 03:59:03.745000-07:00,SAMPLE_SKLEARN_MODEL_3,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"


In [64]:
start_time = time.time()
df =fetch_model("sample_model_9")
end_time = time.time()
print("time taken =", end_time - start_time)
df

time taken = 0.6115124225616455


In [68]:
start_time = time.time()
df = fetch_model_query("sample_model_9")
end_time = time.time()
print("time taken =", end_time - start_time)
df.show()

time taken = 0.4207148551940918
------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"          |"database_name"           |"schema_name"  |"comment"  |"owner"  |"default_version_name"  |"versions"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-06-04 03:45:33.469000-07:00  |SAMPLE_MODEL_7  |MODEL_REGISTRY_SNOWSIGHT  |SNOWINSIGHT    |NULL       |PRAKHAR  |RUN3                    |["RUN3"]    |
------------------------------------------------------------------------------------------------------------------------------------------------------------



In [ ]:
session.sql("LIST STAGES")

In [ ]:
# model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [8]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [9]:
df = registry.show_models()
df
# type(df)

,created_on,name,database_name,schema_name,comment,owner,default_version_name,versions
0,2024-06-04 03:21:29.166000-07:00,SAMPLE_MODEL,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN1,"[""RUN1""]"
1,2024-06-04 04:47:08-07:00,SAMPLE_MODEL_10,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN4,"[""RUN4""]"
2,2024-06-04 03:26:19.863000-07:00,SAMPLE_MODEL_6,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN1,"[""RUN1"",""RUN2""]"
3,2024-06-04 03:45:33.469000-07:00,SAMPLE_MODEL_7,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN3,"[""RUN3""]"
4,2024-06-04 03:47:48.600000-07:00,SAMPLE_MODEL_8,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN3,"[""RUN3"",""RUN4""]"
5,2024-06-04 03:51:57.010000-07:00,SAMPLE_MODEL_9,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN4,"[""RUN4""]"
6,2024-06-05 05:50:06.224000-07:00,SAMPLE_MODEL_TEST_121,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,RUN1,"[""RUN1""]"
7,2024-05-30 03:54:17.224000-07:00,SAMPLE_SKLEARN_MODEL,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
8,2024-05-31 02:58:30.562000-07:00,SAMPLE_SKLEARN_MODEL_2,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"
9,2024-05-31 03:59:03.745000-07:00,SAMPLE_SKLEARN_MODEL_3,MODEL_REGISTRY_SNOWSIGHT,SNOWINSIGHT,None,PRAKHAR,V1,"[""V1""]"


# Get model list from snowpark registry

In [ ]:
model_list = registry.models()
model_list

# Get model details from registry

In [ ]:
# m =registry.get_model("sample_sklearn_model")
m =registry.get_model("sample_model_9")

m

# Get model versions

In [ ]:
df_version = m.show_versions()
df_version

# Set default model version

In [ ]:
# default_version = m.default
m.default = "V1"

# Get default model version details from model-registry

In [ ]:
mv = m.default
mv

In [ ]:
x

In [ ]:
#Load default vesion as python object
# clf = mv.load()

# Run prediction on scikit-learn model

In [ ]:
import numpy as np
import pandas as pd
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)
df = pd.DataFrame(data_list)

In [ ]:


# remote_prediction = mv.run(data_array, function_name="predict")
# remote_prediction = mv.run(X_train.head(1), function_name="predict")
remote_prediction = mv.run(df, function_name="predict")


In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj
output =remote_prediction.to_dict()   # assuming test_features is Snowpark DataFrame
output

In [ ]:
type(X_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

# Deploy model with custom class

In [ ]:
from snowflake.ml.model import custom_model

In [ ]:
class ExamplePipelineModel(custom_model.CustomModel):
#     def __init__(self, context: ModelContext) -> None:
#         super().__init__(context)
#         v = int(open(context.path('config')).read())
#         self.bias = json.loads(v)['bias']

    @custom_model.inference_api
    def run(self, data_list: list) -> pd.DataFrame:
#         features = self.context.model_ref('feature_preproc').transform(input)
#         model_output = self.context.model_ref('m2').predict(
#             self.context.model_ref('m1').predict(features)
#         )
#         return pd.DataFrame({
#             'output': model_output + self.bias})
        data_array = np.asarray(data_list)
        try:
            prediction = model.predict(data_array)
        except:
            prediction = model.predict(data_array.reshape(1, -1))
        return prediction.tolist()

In [ ]:
!pip freeze | grep cloud

# Train & register model

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

In [ ]:
type(y_train)

In [ ]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [ ]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

In [ ]:
# gbm_default.predict(X_train.head(1))
gbm_default.predict(pd.DataFrame(data_list))

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_4")

# Generating model signature

In [ ]:
from snowflake.ml.model import model_signature

In [ ]:
sig = model_signature.infer_signature(
    X_train.head(),
    y_train.head().tolist(),
    input_feature_names=X_train.columns.tolist(),
    output_feature_names=['value_eur'])

In [ ]:
mv = registry.log_model(gbm_default,
                   model_name="sample_sklearn_model__12345",
                   version_name="v1",
                   conda_dependencies=["scikit-learn==1.3.2", "scipy==1.13.1", "cloudpickle==2.2.1"],
                   comment="My awesome ML model",
                    python_version="3.9.19",
                   metrics={"score": 96},
                   sample_input_data=X_train.head()
                       )

In [ ]:
remote_prediction = mv.run(X_train.head(), function_name="predict")
remote_prediction

In [ ]:
import joblib
def dump_model(model, path):
    joblib.dump(model, path)

In [ ]:
dump_model(gbm_default, "/notebooks/notebooks/ml_model_5")

# Create stage in snowflake-model-registry

In [ ]:
# query = """
#         create or replace stage sample_model_serving_udf_stg
#         directory = (enable = true)
#         copy_options = (on_error='skip_file')
#         """

# session.sql(query)



In [ ]:
# To see the stages
session.sql("show stages")

In [ ]:
from snowflake.snowpark.files import SnowflakeFile

In [ ]:
import cloudpickle
with SnowflakeFile.open("@STG_ML_MODELS/ml_model_2") as file:
    m= cloudpickle.load(file)
    print(type(m))

# Upload model file on ML_MODELS stage 

In [ ]:
#create the stage for storing the ML models
# session.sql("USE MODEL_REGISTRY_SNOWSIGHT;")
session.sql('CREATE OR REPLACE STAGE STG_ML_MODELS').show()

In [ ]:
X_train.to_csv("/data/X_train.csv", index = False)

In [ ]:
session.file.put(
    "/data/X_train.csv", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

In [ ]:
session.file.put(
    "/notebooks/notebooks/ml_model_5", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

In [ ]:
session.file.put(
    "/data/ml_model_5", "@STG_ML_MODELS", auto_compress=False, overwrite=True
)

# Read model file from stage

In [ ]:
session.clear_imports()
session.clear_packages()

#Register above uploded model as import of UDF
session.add_import("@STG_ML_MODELS/ml_model_2")

#map packege dependancies
session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")

In [ ]:
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
import snowflake.snowpark.functions as F
from typing import  Optional



def read_file(filename):
    import cloudpickle
    import sys
    import os
    
    #where all imports located at
    import_dir = sys._xoptions.get("snowflake_import_directory")

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = cloudpickle.load(file)
            return m

#register UDF
@F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
def read_model_from_file(file_name:Optional[str]) -> str:
    import numpy as np
    
    # later we will input train data as JSON object
    # hance, we have to convert JSON object as pandas DF
    data_array = np.asarray(data_list)
    pipeline = read_file(file_name)
    print("type of pickle object =", type(pipeline))
#     dict_mod["model_obj"] = pipeline
    return "pipeline"
#     return pipeline.predict(data_array).to_list()




# @F.udf(name = 'READ_MODEL_FROM_FILE', is_permanent = True, replace = True, stage_location = '@STG_ML_MODELS')
# def read_model_from_file(file_name:Optional[str]) -> str:
#     import numpy as np
    
#     # later we will input train data as JSON object
#     # hance, we have to convert JSON object as pandas DF
#     data_array = np.asarray(data_list)
#     pipeline = read_file(file_name)
#     print("type of pickle object =", type(pipeline))
# #     dict_mod["model_obj"] = pipeline
#     return pipeline
# #     return pipeline.predict(data_array).to_list()

In [ ]:
session.sql("DESC FUNCTION READ_MODEL_FROM_FILE("ml_model")").show()

In [ ]:
model_udf = read_model_from_file("ml_model_2")
# model_udf = read_file("ml_model_2")

In [ ]:
model_udf

In [ ]:
import joblib

In [ ]:
# model_obj = joblib.load(read_model_from_file("ml_model_2"))
import os
model_file = os.path.join("/notebooks/notebooks", "ml_model_5")
model_obj = joblib.load(model_file)

In [ ]:
import pandas as pd
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
df = pd.DataFrame(data_list)
remote_prediction = model_obj.predict(df)

In [ ]:
type(remote_prediction)

In [ ]:
X_train.to_csv("/data/X_train.csv")

In [ ]:
import numpy as np

In [ ]:
data_list = [[ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9, 10,  79,  82]]
data_array = np.asarray(data_list)
print("data_array =", data_array)

remote_prediction = model_obj.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction

In [ ]:
remote_prediction = mv.run(X_X_train, function_name="predict")
remote_prediction

In [ ]:
mv.comment

In [ ]:
from snowflake.snowpark import DataFrame

In [ ]:
import pandas as pd

In [ ]:
from snowflake.snowpark.types import Variant
def read_model_from_stage(session: Session, df_dict: dict) -> Variant:

    # print("file_name =", file_name)
    # session.add_import("@STG_ML_MODELS/"+file_name)
    # map packege dependancies
    # session.add_packages("scikit-learn", "pandas", "cloudpickle==2.2.1")



    # from snowflake.snowpark.types import PandasSeries, PandasDataFrame
    # import snowflake.snowpark.functions as F
    # from typing import Optional
    from snowflake.snowpark.files import SnowflakeFile
    from snowflake.ml.registry.registry import Registry

    import joblib
    df = pd.DataFrame(df_dict)
    print("df ===", df)

    model_file = '@STG_ML_MODELS/ml_model_5'
    # Specify 'mode = rb' to open the file in binary mode.
    with SnowflakeFile.open(model_file, 'rb', require_scoped_url=False) as f:
        model_obj = joblib.load(f)
        # return str(model_obj)
        # return str(model_obj)

        reg = Registry(session=session)

        mv = reg.log_model(model=model_obj,
                           model_name="sample_model",
                           comment="test",
                           version_name="run1",
                           python_version="3.9.19",
                           conda_dependencies=["scikit-learn==1.3.2"],
                           metrics={"model_metrics": {"score": 96}, "project_id": "0001", "type": "Model"},
                           sample_input_data=df
                           )

        print("model got registered successfully")
        return df_dict

In [ ]:
session.sproc.register(func=read_model_from_stage,
                           name="read_model_from_stage",
                           packages=["snowflake-snowpark-python", "snowflake-ml-python","scikit-learn", "pandas", "cloudpickle==2.2.1", "joblib"],
                           isPermanant=True,
                           stage_location="@STG_ML_MODELS",
                           replace=True,
                       )

In [ ]:
dict_df = X_train.head().to_dict()

In [ ]:
dict_df

In [ ]:
X_train.columns

In [ ]:
df = pd.DataFrame(dict_df)

In [ ]:
df.columns

In [ ]:
model_obj = session.call("read_model_from_stage", dict_df)

In [ ]:
!pip freeze | grep -i categ

In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
# import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
# from refractml import *
# from refractml.constants import MLModelFlavours

In [ ]:
cars = pd.read_csv("/data/car_new.csv")
cars

In [ ]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

In [ ]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [ ]:
model.predict(X_train.head(1))

In [ ]:
from snowflake.ml.model import model_signature

In [ ]:
y_train.head().tolist()

In [ ]:
sig = model_signature.infer_signature(
    X_train.head(),
    y_train.head().tolist(),
    input_feature_names=X_train.columns.tolist(),
    output_feature_names=['class_num'])

# 